In [44]:
# %run ../imports.ipynb
from tqdm.auto import tqdm
import plotly.express as px
from IPython.core.display import display, HTML
import pandas as pd
import time
import warnings
from requests import get
from ratelimit import limits
import numpy as np
import itertools
pd.set_option('max_columns',1000)

import requests
import pickle
from scipy import stats
import matplotlib.dates as dates
import datetime, calendar
import scipy.stats


In [2]:
with open('catmap.pickle', 'rb') as handle:
    cat_map = pickle.load(handle)

In [3]:
with open('df_prices.pickle', 'rb') as handle:
    df_prices = pickle.load(handle)

In [4]:
df_prices_21=df_prices[df_prices.time>='20210101']
df_prices_21=df_prices_21[~pd.isnull(df_prices_21)]

In [5]:
df_prices_21.shape

(111809, 7)

In [6]:
df_prices_21['30_d_pct_chg']=df_prices_21.groupby(['coin_id'])['prices'].pct_change(periods=30)

In [45]:
def get_poly_r2(df, deg=3):
    x=dates.date2num(df.time)
    poly=np.polyfit(x, df.prices, deg)
    slope, intercept, r_value, p_value, std_err = stats.linregress(np.polyval(poly, x), df.prices)
    return r_value**2


In [63]:
str(df_prices_21.tail(1).time.iloc[0])

'2021-12-10 19:00:00'

In [46]:
get_poly_r2(df_prices_21.head(30))

0.943413478545005

In [ ]:
g=df_prices_21.groupby('coin_id').rolling(window=30)
lin_accum_r2=[]
for name, group in df_prices_21.groupby('coin_id'):
    for w in group.rolling(30):
        r2=get_poly_r2(w)
        lin_accum_r2.append({'coin_id':name,'time':w.tail(1).time.iloc[0],'30d_r2':r2})


In [67]:
lin_accum_r2_df=pd.DataFrame(lin_accum_r2)

In [74]:
df_prices_21_r2=df_prices_21.merge(lin_accum_r2_df)

In [86]:
df_30_lin=df_prices_21_r2[(df_prices_21_r2['30_d_pct_chg']>1)&(df_prices_21_r2['30d_r2']>.9)]

In [93]:
df_30_lin['consec']=df_30_lin['time'].shift(1)==df_30_lin['time']-datetime.timedelta(days=1)

/tmp/ipykernel_27349/1775414698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30_lin['consec']=df_30_lin['time'].shift(1)==df_30_lin['time']-datetime.timedelta(days=1)


In [100]:
df_30_lin.head().consec

30    False
31     True
32     True
37    False
38     True
Name: consec, dtype: bool

In [108]:
df_30_lin['cons_shift']=df_30_lin.consec.shift(-1).fillna(True)

/tmp/ipykernel_27349/3458282862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30_lin['cons_shift']=df_30_lin.consec.shift(-1).fillna(True)


maybe only take first instance of sequence as start point to plot ( consec=false, no falses within 30 days in the future)

In [161]:
df_30_lin_filt=df_30_lin[(~df_30_lin.consec)|(~df_30_lin.cons_shift)]
df_30_lin_filt=df_30_lin[(~df_30_lin.consec)]#(~df_30_lin.consec)|

In [178]:
cnt=df_30_lin_filt.groupby('coin_id').size().reset_index()
cnt=cnt[cnt[0]>1]


In [179]:
sing=df_30_lin[~df_30_lin.coin_id.isin(cnt.coin_id)]
mult=df_30_lin[df_30_lin.coin_id.isin(cnt.coin_id)]

In [186]:
mult['dist_from_max']=mult.groupby('coin_id')['time'].transform('max') - mult['time']
mult['dist_from_min']=mult.groupby('coin_id')['time'].transform('min') - mult['time']

/tmp/ipykernel_27349/1767811335.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult['dist_from_max']=mult.groupby('coin_id')['time'].transform('max') - mult['time']
/tmp/ipykernel_27349/1767811335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult['dist_from_min']=mult.groupby('coin_id')['time'].transform('min') - mult['time']


In [216]:
def is_isolated(df):
    iso=[]
    for time in df.time:
        end=time-datetime.timedelta(days=45)
        is_iso=(df[(df.time<time)&(df.time>=end)].shape[0]<1)
        iso.append(is_iso)
    df['is_iso']=iso
    return(df)


In [217]:
is_iso=[]
for name, group in df_30_lin.groupby('coin_id'):
    
    is_iso.append(is_isolated(group))


In [268]:
plot_markers=pd.concat(is_iso)
plot_markers=plot_markers[plot_markers.is_iso]
plot_markers=plot_markers[['time','coin_id']].drop_duplicates()
plot_markers['disp']=plot_markers['time'].map(lambda x: str(x.date()))
plot_markers['disp']=plot_markers['coin_id']+'-'+plot_markers['disp']


In [265]:
dfl=[]
for index, row in plot_markers.iterrows():
    end=row.time+datetime.timedelta(days=60)

    tmp=df_prices_21[(df_prices_21.coin_id==row.coin_id)&(df_prices_21.time>=row.time)&(df_prices_21.time<end)].reset_index(drop=True).reset_index()
    tmp['pct_chg_from_start']=tmp.groupby('coin_id')['prices'].apply(lambda x: x.div(x.iloc[0]).subtract(1).mul(100))
    tmp['disp']=str(tmp['time'].min().date())
    tmp['disp']=tmp['coin_id']+'-'+tmp['disp']

    dfl.append(tmp)


In [266]:
df_final=pd.concat(dfl)

In [270]:
fig = px.line(df_final, x='index', y='pct_chg_from_start',
              color="disp",
              line_group="disp", hover_name="disp")
fig.update_layout(title='60 days after linear accumulation signal' , showlegend=True, legend_traceorder='normal')


fig.show()


In [129]:
dt='2021-01-31 19:00:00'

In [139]:
pd.to_datetime(1612119600000000000)#-datetime.timedelta(days=1)

Timestamp('2021-01-31 19:00:00')

In [128]:
pd.to_datetime(1612119600000000000)
df_30_lin_filt[df_30_lin_filt.time=='2021-01-31 19:00:00']

,time,coin_id,prices,market_caps,total_volumes,price_pct_change,vol_over_market_cap,30_d_pct_chg,30d_r2,consec,cons_shift
30,2021-01-31 19:00:00,avalanche-2,13.469298,1.036012e+09,3.029140e+08,154.895437,0.292384,2.674273,0.923210,False,True
32,2021-02-02 19:00:00,avalanche-2,13.504583,1.045592e+09,8.069360e+07,155.563184,0.077175,2.902583,0.905581,True,False
37,2021-02-07 19:00:00,avalanche-2,24.087305,1.787279e+09,4.698480e+08,355.832540,0.262885,3.146693,0.905723,False,True
39,2021-02-09 19:00:00,avalanche-2,31.596744,2.448785e+09,3.694652e+08,497.942525,0.150877,3.789698,0.958463,True,False
41,2021-02-11 19:00:00,avalanche-2,47.228492,3.640061e+09,6.748677e+08,793.760547,0.185400,5.092240,0.923666,False,True
...,...,...,...,...,...,...,...,...,...,...,...
111681,2021-11-22 19:00:00,airswap,0.550246,9.470581e+07,8.361622e+06,121.964082,0.088290,1.167267,0.913778,False,True
111684,2021-11-25 19:00:00,airswap,0.570160,9.839493e+07,3.239365e+06,129.997067,0.032922,1.285108,0.918928,True,False
111779,2021-11-11 19:00:00,neos-credits,0.478202,7.993636e+06,1.436650e+05,169.001279,0.017972,1.084928,0.904183,False,True
111781,2021-11-13 19:00:00,neos-credits,0.600743,1.067869e+07,1.662364e+02,237.933937,0.000016,1.290336,0.919349,True,False
